#EKSPERIMEN LSTM FRAMEWORK NLP PyTorch Menggunakan Word Embedding GLOVE

###data preprocessing



In [ ]:
!pip install torch
import torch

In [ ]:
!pip install torchtext

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Activation, Dropout, Flatten
from tensorflow.keras.initializers import Constant
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize

from sklearn import metrics
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from subprocess import check_output
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, re, csv, math, codecs

In [ ]:
# Fungsi untuk membaca file dataset
def load_dataset(filepath):
    texts = []
    labels = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            split_line = line.strip().split('\t')
            if len(split_line) == 2:  # Pastikan format benar
                texts.append(split_line[0])
                try:
                    labels.append(int(split_line[1]))  # Konversi label menjadi integer
                except ValueError:
                    print(f"Label tidak valid pada baris: {line}")
    return texts, labels

# Load train dan test dataset
df1 = pd.read_csv("/content/test.csv")
df2 = pd.read_csv("/content/train.csv")


In [ ]:
df1.head(5)

,3,Fears for T N pension after talks,Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
0,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
1,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
2,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
3,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
4,4,Open Letter Against British Copyright Indoctri...,The British Department for Education and Skill...


In [ ]:
df2.head(5)

,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...


In [ ]:
# Print column names to identify the correct ones
print("Columns in df1:", df1.columns)
print("Columns in df2:", df2.columns)

Columns in df1: Index(['3', 'Fears for T N pension after talks',
       'Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.'],
      dtype='object')
Columns in df2: Index(['3', 'Wall St. Bears Claw Back Into the Black (Reuters)',
       'Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.'],
      dtype='object')


In [ ]:
text_column_name_train = df2.columns[2]
label_column_name_train = 'Fears for T N pension after talks'

text_column_name_test = df1.columns[1]
label_column_name_test = 'Wall St. Bears Claw Back Into the Black (Reuters)'

In [ ]:
text_column_name_train = df2.columns[2]
label_column_name_train = df2.columns[1]

text_column_name_test = df1.columns[1]
label_column_name_test = df1.columns[2]

def load_dataset_from_df(df, text_column, label_column):
    """Memuat dataset dari pandas DataFrame.

    Args:
        df: pandas DataFrame yang berisi data.
        text_column: Nama kolom yang berisi data teks.
        label_column: Nama kolom yang berisi label data.

    Returns:
        texts: Daftar string teks.
        labels: Daftar label yang sesuai.
    """
    texts = df[text_column].tolist()
    labels = df[label_column].tolist()
    return texts, labels

train_texts, train_labels = load_dataset_from_df(df2, text_column_name_train, label_column_name_train)
test_texts, test_labels = load_dataset_from_df(df1, text_column_name_test, label_column_name_test)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(train_texts)

###GLOVE

In [ ]:
EMBEDDING_DIM = 100
glove_path = "glove.6B.100d.txt"
MAX_VOCAB_SIZE = 10000 # Define MAX_VOCAB_SIZE here or any desired size

# Membaca file GloVe
embeddings_index = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((MAX_VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float)

In [ ]:
df2.columns = df2.columns.str.strip()  # Menghapus spasi di awal dan akhir kolom
df1.columns = df1.columns.str.strip()  # Menghapus spasi di awal dan akhir kolom

print("Kolom di df2:", df2.columns)
print("Kolom di df1:", df1.columns)

text_column_name_train = df2.columns[2]
label_column_name_train = df2.columns[1]

text_column_name_test = df1.columns[1]
label_column_name_test = df1.columns[2]


def load_dataset_from_df(df, text_column, label_column):
    """Memuat dataset dari pandas DataFrame.

    Args:
        df: pandas DataFrame yang berisi data.
        text_column: Nama kolom yang berisi data teks.
        label_column: Nama kolom yang berisi label data.

    Returns:
        texts: Daftar string teks.
        labels: Daftar label yang sesuai.
    """
    texts = df[text_column].tolist()
    labels = df[label_column].tolist()
    return texts, labels

# Memuat dataset untuk data pelatihan dan pengujian menggunakan fungsi yang sudah diperbarui
train_texts, train_labels = load_dataset_from_df(df2, text_column_name_train, label_column_name_train)
test_texts, test_labels = load_dataset_from_df(df1, text_column_name_test, label_column_name_test)

Kolom di df2: Index(['3', 'Wall St. Bears Claw Back Into the Black (Reuters)',
       'Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.'],
      dtype='object')
Kolom di df1: Index(['3', 'Fears for T N pension after talks',
       'Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.'],
      dtype='object')


###LSTM

In [ ]:
import torch.nn as nn

class YourModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(YourModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, 128)  # Embedding layer
        self.lstm = nn.LSTM(128, 128, batch_first=True)  # LSTM layer
        self.fc = nn.Linear(128, output_dim)  # Output layer

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)  # LSTM output
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [ ]:
import torch.nn as nn
import torch.optim as optim
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout):
        super(LSTMClassifier, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        x = self.fc(self.dropout(hidden[-1]))
        return x

# Parameter model
HIDDEN_DIM = 128
OUTPUT_DIM = 2
DROPOUT = 0.5

model = LSTMClassifier(embedding_matrix, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

def train(model, loader):
    model.train()
    total_loss = 0
    for sequences, labels in loader:
        sequences = sequences.long()
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for sequences, labels in loader:
            sequences = sequences.long()
            output = model(sequences)
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

train_sequences = torch.randint(0, 10, (1000, 128))
train_labels = torch.randint(0, 2, (1000,))

test_sequences = torch.randint(0, 10, (200, 128))
test_labels = torch.randint(0, 2, (200,))

train_dataset = TensorDataset(train_sequences, train_labels)
test_dataset = TensorDataset(test_sequences, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
EPOCHS = 25
for epoch in range(EPOCHS):
    train_loss = train(model, train_loader)
    test_acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Epoch 1/25, Loss: 0.7886, Test Accuracy: 0.5050
Epoch 2/25, Loss: 0.7095, Test Accuracy: 0.4900
Epoch 3/25, Loss: 0.6881, Test Accuracy: 0.5000
Epoch 4/25, Loss: 0.6881, Test Accuracy: 0.4900
Epoch 5/25, Loss: 0.6780, Test Accuracy: 0.4650
Epoch 6/25, Loss: 0.6633, Test Accuracy: 0.5000
Epoch 7/25, Loss: 0.6678, Test Accuracy: 0.4650
Epoch 8/25, Loss: 0.6528, Test Accuracy: 0.4500
Epoch 9/25, Loss: 0.6413, Test Accuracy: 0.4600
Epoch 10/25, Loss: 0.6312, Test Accuracy: 0.4900
Epoch 11/25, Loss: 0.6022, Test Accuracy: 0.4750
Epoch 12/25, Loss: 0.5963, Test Accuracy: 0.5050
Epoch 13/25, Loss: 0.5815, Test Accuracy: 0.4750
Epoch 14/25, Loss: 0.5672, Test Accuracy: 0.5050
Epoch 15/25, Loss: 0.5454, Test Accuracy: 0.5300
Epoch 16/25, Loss: 0.5254, Test Accuracy: 0.5000
Epoch 17/25, Loss: 0.5122, Test Accuracy: 0.5350
Epoch 18/25, Loss: 0.5255, Test Accuracy: 0.4550
Epoch 19/25, Loss: 0.4849, Test Accuracy: 0.5250
Epoch 20/25, Loss: 0.4584, Test Accuracy: 0.4900
Epoch 21/25, Loss: 0.4362, Te

In [ ]:
#Evaluasi Model
final_accuracy = evaluate(model, test_loader)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Final Test Accuracy: 0.5150
